In [39]:
%load_ext autoreload
%autoreload 2
import logging
import unittest
import warnings
from pathlib import Path
from sqlalchemy import MetaData

import pytest
from faker import Faker

import cspawn
from cspawn.cli.util import create_demo_users, create_demo_images, make_data
from cspawn.docker.models import CodeHost, HostImage
from cspawn.init import db
from cspawn.main.models import User
from cspawn.apptypes import App 
from cspawn.cli.util import create_demo_users, create_demo_images, make_data
from cspawn.test_fixture import make_fake_user  

from cspawn.init import init_app

logger = logging.getLogger("cspawn.docker")
logger.setLevel(logging.INFO)

this_dir = Path().cwd()
config_dir = Path(cspawn.__file__).parent.parent
dev_root = this_dir.parent
data_dir = dev_root / "data"

warnings.filterwarnings("ignore")

app = init_app(
    config_dir=config_dir,
    log_level=logging.ERROR,
    sqlfile=this_dir / "test.db")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:

with app.app_context():
    print(app.db.engine.url)
    
    db = app.db
    e = db.engine

    m = MetaData()
    m.reflect(e)
    m.drop_all(e)
    print("Database tables destroyed successfully.")

    db.create_all()

    create_demo_users(app)
    create_demo_images(app)

with app.app_context():
    app.csm.sync(check_ready=True)    


sqlite:////Users/eric/proj/league-projects/code-server-mono/code-server-spawner/test/test.db
Database tables destroyed successfully.


INFO:cspawn.docker:Services in db but not in swarm: 0
INFO:cspawn.docker:Syncing not-ready hosts: 0
INFO:cspawn.docker:Syncing not-in-db hosts: 0


In [4]:
fake = Faker()

with app.app_context():
    users = [make_fake_user(fake) for _ in range(100)]
    app.db.session.bulk_save_objects(users)
    app.db.session.commit()

In [28]:
from time import sleep, time
from IPython.display import display
from IPython.display import clear_output

with app.app_context():
    print(app.db.engine.url)
    users = User.query.all()
    user: User = fake.random_element(users)
    image: HostImage = HostImage.query.first()

    csm = app.csm

    r = csm.new_cs(user, image.image_uri, image.repo_uri)

    e = str(user), r.name, r.container_states

    while True:
        if r.check_ready():
            break
        rec = r.rec
        clear_output(wait=True)
        display(f"{time()} {rec.service_name} {rec.state}  {rec.app_state}")
        sleep(2)
        
    url = r.rec.public_url
    



'1740879958.368096 solomonjennifer running  unknown'

In [29]:
with app.app_context():
    r.sync_to_db(check_ready=True)

In [8]:
with app.app_context():
    x = app.csm.get('u8cmauoe5bb7xvu6qy5zc72xo')
    x.sync_to_db()
    


AttributeError: 'NoneType' object has no attribute 'sync_to_db'

In [ ]:
x.id

'u8cmauoe5bb7xvu6qy5zc72xo'

In [ ]:
r.hostname

'bobstaff.doswarm.jointheleague.org'